In [1]:
import json
from shapely.geometry import Polygon
import numpy as np
import pyproj

In [2]:
with open("data/aoi.geojson", "r") as fp:
    aoi = json.load(fp)

In [3]:
x0 = aoi['features'][0]['geometry']['coordinates'][0][0][0][0]
y0 = aoi['features'][0]['geometry']['coordinates'][0][0][0][1]
aoi_width = aoi['features'][0]['geometry']['coordinates'][0][0][1][0] - aoi['features'][0]['geometry']['coordinates'][0][0][0][0]
aoi_height = aoi['features'][0]['geometry']['coordinates'][0][0][0][1] - aoi['features'][0]['geometry']['coordinates'][0][0][2][1]

In [4]:
chip_dim = 224 #pixels
chip_resolution = 30 #meters
chip_width = chip_dim * chip_resolution
chip_height = chip_dim * chip_resolution

In [5]:
features = []
proj = pyproj.Transformer.from_crs(int(aoi['crs']['properties']['name'].split("::")[-1]), 4326, always_xy=True)

for i_x in range(0, int(np.floor(aoi_width/chip_width))):
    for i_y in range(0, int(np.floor(aoi_height/chip_height))):
        feature_template = [{'type': 'Feature',
                    'properties': {'id': 0,
                                  'center': [0, 0]},
                    'geometry': {'type': 'MultiPolygon',
                    'coordinates': [[[[-281415, 1533615],
                                      [256185, 1533615],
                                      [256185, 996015],
                                      [-281415, 996015],
                                      [-281415, 1533615]]]]
                                }
                   }]
        chip_id = f"{str(i_x).zfill(3)}_{str(i_y).zfill(3)}"
        chip_x0 = x0 + i_x * chip_width
        chip_y0 = y0 - i_y * chip_height
        coor = [[[[proj.transform(chip_x0, chip_y0)[0], proj.transform(chip_x0, chip_y0)[1]],
                  [proj.transform(chip_x0, chip_y0 - chip_height)[0], proj.transform(chip_x0, chip_y0 - chip_height)[1]],
                  [proj.transform(chip_x0 + chip_width, chip_y0 - chip_height)[0], proj.transform(chip_x0 + chip_width, chip_y0 - chip_height)[1]],
                  [proj.transform(chip_x0 + chip_width, chip_y0)[0], proj.transform(chip_x0 + chip_width, chip_y0)[1]],
                  [proj.transform(chip_x0, chip_y0)[0], proj.transform(chip_x0, chip_y0)[1]]
            ]]]
        chip_cen = [chip_x0 + chip_width / 2, chip_y0 - chip_height / 2]
        chip_cen_lon, chip_cen_lat = proj.transform(chip_cen[0], chip_cen[1])
        
        feature_template[0]['geometry']['coordinates'] = coor
        feature_template[0]['properties']['id'] = chip_id
        feature_template[0]['properties']['center'] = [chip_cen_lon, chip_cen_lat] 
        features.extend(feature_template)

feature_collection = {
    'type': 'FeatureCollection',
     'name': 'chips',
     'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}},
     'features': features
}

In [6]:
with open("data/chip_bbox.geojson", 'w') as outfile:
    json.dump(feature_collection, outfile)